# Procesos de Decisión de Markov

En los Procesos de Decisión de Markov (PDMs) encadenamos la decisiones del agente con un proceso estocástico.

Nos interesa resolver la ecuación de Bellman para la política del agente.

Recordemos que la política le dice al agente que acción tomar en cada estado posible.

Para encontrar la politica optima $\pi^*$ podemos usar el algoritmo de iteración de políticas.

A continuación te damos una implementación básica del algoritmo.

In [1]:
import numpy as np
from itertools import product
from random import choice
class MDP:
    
    def __init__(self,s,r,a,T,gamma):
        """
        Builds the MDP problem
        :param s: states
        :param r: rewards
        :param a: actions
        :param T: dictionary where keys are (s,a) pairs and
        values are probabilities
        :param gamma: the discount factor
        """
        self.s = s
        self.r = r
        self.a = a
        self.T = T
        self.gamma = gamma
        
    def policy_iteration(self,pi=None):
        """
            Policy iteration algorithm
        """
        #initial random policy
        if not pi:
            pi = [choice(self.a) for s in self.s]
        
        print('pi = '+str(pi))
        self.obtainT(pi);
        T = self.obtainT(pi)
        print('T(pi) = \n'+str(T))
        
        V = np.matmul(np.linalg.inv(np.eye(3)-self.gamma*T.T),self.r)
        print('V(s) = '+ str(V))
        
        pi_star = self.find_pi_star(V)
        print("pi*(s) = "+str(pi_star))
        
        if pi_star == pi:
            return pi
        else:
            return self.policy_iteration(pi_star)
        
    def obtainT(self,pi):
        """
        Obtains the transition probability matrix parametrized by the policy pi
        :param pi: the policy
        """
        return \
          np.matrix([[self.T[(s,t,pi[s])] for s in self.s] for t in self.s])
        
    def find_pi_star(self,V):
        """
        Finds the optimal policy for the given infinite horizon values
        :param V: the infinite horizon expected utility
        """
        
        print("\n".join([str((x,np.matmul(self.obtainT(x).T,np.array(V).T))) for x in product(self.a,self.a,self.a)]))
        
        return list(max(product(self.a,self.a,self.a),\
            key=lambda x: np.sum(np.matmul(self.obtainT(x).T,np.array(V).T))))
            
    

Para ejecutar el algoritmo primero definimos los estados posibles:

In [2]:
estados = [0,1,2]
print(estados)

[0, 1, 2]


Las acciones posibles:

In [3]:
acciones = [0,1]
print(acciones)

[0, 1]


Ahora las recompensas para cada estado:

In [4]:
recompensas = [0,10,27]
print(recompensas)

[0, 10, 27]


El parámetro gamma es el factor de descuento para la utilidad de valores futuros:

In [5]:
gamma = 0.9
print(gamma)

0.9


Ahora representamos las probabilidaddes condicionales con un diccionario.

<img src="mdpejemplo2.png" alt="mdp" width="914"/>

La llave será una tupla con tres elementos, los primeros dos contienen los índices de la transición de estado, el último es la acción que se toma.

El valor del diccionario es la probabilidad asociada al estado, acción correspondiente.

In [6]:
T={
    (0,0,0):0.7,(0,0,1):0.5, (1,0,0):0.4,(1,0,1):0.2, (2,0,0):0.2,(2,0,1):0.1,
    (0,1,0):0.1,(0,1,1):0.3, (1,1,0):0.4,(1,1,1):0.7, (2,1,0):0.2,(2,1,1):0.1,
    (0,2,0):0.2,(0,2,1):0.2, (1,2,0):0.2,(1,2,1):0.1, (2,2,0):0.6,(2,2,1):0.8
}
print(T)

{(0, 0, 0): 0.7, (0, 0, 1): 0.5, (1, 0, 0): 0.4, (1, 0, 1): 0.2, (2, 0, 0): 0.2, (2, 0, 1): 0.1, (0, 1, 0): 0.1, (0, 1, 1): 0.3, (1, 1, 0): 0.4, (1, 1, 1): 0.7, (2, 1, 0): 0.2, (2, 1, 1): 0.1, (0, 2, 0): 0.2, (0, 2, 1): 0.2, (1, 2, 0): 0.2, (1, 2, 1): 0.1, (2, 2, 0): 0.6, (2, 2, 1): 0.8}


Vamos a crear una instancia del PDM:

In [7]:
mdp = MDP(estados,recompensas,acciones,T,gamma)

Comprobamos que obtenemos la misma política que el ejemplo del tren inteligente.

En el ejemplo $\pi_0 = \begin{bmatrix}0&0&0\end{bmatrix}$.

In [8]:
pi_0 = [0,0,0]
print("T(pi_0) = \n"+str(mdp.obtainT(pi_0)))

T(pi_0) = 
[[0.7 0.4 0.2]
 [0.1 0.4 0.2]
 [0.2 0.2 0.6]]


Ahora invocamos el algoritmo de iteración de políticas.

El algoritmo imprime como cambia la política con las iteraciones.

In [9]:
politica = mdp.policy_iteration(pi_0)

pi = [0, 0, 0]
T(pi) = 
[[0.7 0.4 0.2]
 [0.1 0.4 0.2]
 [0.2 0.2 0.6]]
V(s) = [[ 91.73373288 105.43236301 135.84760274]]
((0, 0, 0), matrix([[101.92636986],
        [106.0359589 ],
        [120.94178082]]))
((0, 0, 1), matrix([[101.92636986],
        [106.0359589 ],
        [128.39469178]]))
((0, 1, 0), matrix([[101.92636986],
        [105.73416096],
        [120.94178082]]))
((0, 1, 1), matrix([[101.92636986],
        [105.73416096],
        [128.39469178]]))
((1, 0, 0), matrix([[104.66609589],
        [106.0359589 ],
        [120.94178082]]))
((1, 0, 1), matrix([[104.66609589],
        [106.0359589 ],
        [128.39469178]]))
((1, 1, 0), matrix([[104.66609589],
        [105.73416096],
        [120.94178082]]))
((1, 1, 1), matrix([[104.66609589],
        [105.73416096],
        [128.39469178]]))
pi*(s) = [1, 0, 1]
pi = [1, 0, 1]
T(pi) = 
[[0.5 0.4 0.1]
 [0.3 0.4 0.1]
 [0.2 0.2 0.8]]
V(s) = [[127.58241758 138.57142857 181.97802198]]
((0, 0, 0), matrix([[139.56043956],
        [142.8571

## Tarea

Si cambiamos el valor las recompensas a  [0,20,20] , ¿cambia la politica?

In [10]:
#cambia las recompensas y ejecuta la siguiente línea. 
#politica = mdp.policy_iteration([0,1,0])

#copia el resultado en un archivo y sométe la respuesta.